In [25]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config
from botocore.exceptions import NoCredentialsError
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import mapping
from datetime import datetime, timedelta

In [26]:
# Create an unsigned S3 client
s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
bucket_name = "sentinel-cogs"
response = s3_client.list_objects_v2(Bucket='sentinel-cogs', Prefix='sentinel-s2-l2a-cogs/2024-01-01')
for obj in response.get('Contents', []):
    print(obj['Key'])

In [27]:
# Function to find Sentinel-2 imagery based on date and fallback to latest if not available
def find_sentinel_image(input_date):
    prefix = f"sentinel-s2-l2a-cogs/{input_date[:4]}/{input_date[5:7]}/{input_date[8:10]}/"
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

    # If data exists for the requested date
    if 'Contents' in response:
        for obj in response['Contents']:
            if obj['Key'].endswith('B08.tif'):  # Look for Band 8 (NIR)
                return obj['Key']

    # Fallback to latest available image
    print(f"No data found for {input_date}. Searching for the latest available image.")
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix="sentinel-s2-l2a-cogs/")
    if 'Contents' in response:
        for obj in response['Contents']:
            if obj['Key'].endswith('B08.tif'):
                return obj['Key']

    return None

In [28]:
# Function to download Sentinel-2 imagery
def download_image(key, file_path):
    try:
        s3_client.download_file(bucket_name, key, file_path)
    except NoCredentialsError:
        raise Exception("AWS credentials not found. Ensure they are correctly configured.")


In [29]:
# Function to calculate NDMI
def calculate_ndmi(nir_path, swir_path, aoi_polygon):
    with rasterio.open(nir_path) as nir_src, rasterio.open(swir_path) as swir_src:
        # Mask and read the NIR band
        nir_data, nir_transform = mask(nir_src, [mapping(aoi_polygon)], crop=True)
        nir_data = nir_data[0].astype('float32')

        # Mask and read the SWIR band
        swir_data, swir_transform = mask(swir_src, [mapping(aoi_polygon)], crop=True)
        swir_data = swir_data[0].astype('float32')

        # Calculate NDMI
        ndmi = (nir_data - swir_data) / (nir_data + swir_data)

        # Compute mean NDMI value
        mean_ndmi = np.nanmean(ndmi)

        return ndmi, mean_ndmi

In [30]:
# Function to save NDMI image as PNG
def save_ndmi_image(ndmi, output_path):
    plt.figure(figsize=(10, 10))
    plt.imshow(ndmi, cmap='RdYlGn', vmin=-1, vmax=1)
    plt.colorbar(label="NDMI")
    plt.title("Normalized Difference Moisture Index (NDMI)")
    plt.axis('off')
    plt.savefig(output_path, bbox_inches='tight')
    plt.close()

In [31]:
def calculate_ndmi(nir_path, swir_path, aoi_polygon):
    with rasterio.open(nir_path) as nir_src, rasterio.open(swir_path) as swir_src:
        # Reproject AOI polygon to match raster CRS
        raster_crs = nir_src.crs
        if farm_polygon.crs != raster_crs:
            farm_polygon = farm_polygon.to_crs(raster_crs)

        # Check if AOI overlaps with raster bounds
        if not farm_polygon.intersects_box(*nir_src.bounds):
            raise ValueError("AOI polygon does not overlap with the raster extent.")

        # Mask and read the NIR band
        nir_data, _ = mask(nir_src, [mapping(aoi_polygon)], crop=True)
        nir_data = nir_data[0].astype('float32')

        # Mask and read the SWIR band
        swir_data, _ = mask(swir_src, [mapping(aoi_polygon)], crop=True)
        swir_data = swir_data[0].astype('float32')

        # Calculate NDMI
        ndmi = (nir_data - swir_data) / (nir_data + swir_data)

        # Compute mean NDMI value
        mean_ndmi = np.nanmean(ndmi)

        return ndmi, mean_ndmi


In [32]:
# Example usage
if __name__ == "__main__":
    farm_polygon_path = "D:\IIST-MTECH\CaseStudy_VL\map.geojson"  # Replace with your actual file path
    input_date = "2022-10-10"  # Desired date (YYYY-MM-DD)
    output_image_path = "ndmi_output.png"

    mean_ndmi, result = calculate_ndmi(farm_polygon_path, input_date, output_image_path)

    if mean_ndmi is not None:
        print(f"NDMI image saved at {result}. Mean NDMI value: {mean_ndmi}")
    else:
        print(f"Error: {result}")


RasterioIOError: 'D:/IIST-MTECH/CaseStudy_VL/map.geojson' not recognized as a supported file format.